In [ ]:
import networkx as nx
import numpy as np
import random
import time
import matplotlib.pyplot as plt

def initialize_network(N, M, initial_defector_ratio):
    """
    Initialize a random graph with N nodes and M edges, assigning 'C' (cooperator) or 'D' (defector) states.
    Args:
        N (int): Number of nodes in the graph.
        M (int): Number of edges in the graph.
        initial_defector_ratio (float): Initial fraction of nodes to be defectors.
    Returns:
        G (networkx.Graph): Initialized graph with nodes assigned states.
    """
    G = nx.gnm_random_graph(N, M)
    num_defectors = int(N * initial_defector_ratio)
    states = ['D'] * num_defectors + ['C'] * (N - num_defectors)
    random.shuffle(states)
    for node, state in zip(G.nodes(), states):
        G.nodes[node]['state'] = state
    return G

def update_strategy(G, alpha, u, discordant_edges):
    """
    Update the state of a node along a discordant edge based on payoff difference.
    Args:
        G (networkx.Graph): The graph with nodes and their states.
        alpha (float): Sensitivity parameter for strategy update.
        u (float): Cost-to-benefit ratio.
        discordant_edges (list): List of edges where nodes have different states.
    Returns:
        discordant_edges (list): Updated list of discordant edges.
    """
    if discordant_edges:
        edge = random.choice(discordant_edges)
        c_node, d_node = edge if G.nodes[edge[0]]['state'] == 'C' else (edge[1], edge[0])
        payoff1 = sum(1 if G.nodes[n]['state'] == 'C' else 0 for n in G.neighbors(c_node))
        payoff2 = sum(1 + u if G.nodes[n]['state'] == 'C' else u for n in G.neighbors(d_node))
        probability_1_to_2 = 1 / (1 + np.exp(alpha * (payoff1 - payoff2))) if (payoff1 - payoff2) <= 2 else 0
        if random.random() < probability_1_to_2:
            G.nodes[c_node]['state'] = 'D'
            changed_node = c_node
        else:
            G.nodes[d_node]['state'] = 'C'
            changed_node = d_node
        discordant_edges.remove(edge)
        for neighbor in G.neighbors(changed_node):
            potential_edge = tuple(sorted([changed_node, neighbor]))
            if G.nodes[changed_node]['state'] != G.nodes[neighbor]['state'] and potential_edge not in discordant_edges:
                discordant_edges.append(potential_edge)
            elif G.nodes[changed_node]['state'] == G.nodes[neighbor]['state'] and potential_edge in discordant_edges:
                discordant_edges.remove(potential_edge)
    return discordant_edges

def rewire(G, discordant_edges, strategy_type, p, threshold):
    """
    Rewire a discordant edge based on the specified strategy, updating the discordant edge list.
    Args:
        G (networkx.Graph): The graph with nodes and their states.
        discordant_edges (list): List of edges where nodes have different states.
        strategy_type (str): Rewiring strategy ('max_degree_C', 'min_degree_C', 'probabilistic', 'random', 'hybrid').
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy to switch between max_degree_C and min_degree_C.
    Returns:
        discordant_edges (list): Updated list of discordant edges.
    """
    if discordant_edges:
        edge = random.choice(discordant_edges)
        c_node, d_node = edge if G.nodes[edge[0]]['state'] == 'C' else (edge[1], edge[0])
        G.remove_edge(*edge)
        discordant_edges.remove(edge)
        potential_nodes = [node for node in G.nodes() if not G.has_edge(c_node, node) and node != c_node]
        new_node = None
        if strategy_type == 'hybrid':
            coop_fraction = calculate_cooperation_fraction(G)
            if coop_fraction < threshold:
                strategy_type = 'max_degree_C'
            else:
                strategy_type = 'min_degree_C'

        if strategy_type == 'max_degree_C':
            max_degree = -1
            for node in potential_nodes:
                if G.nodes[node]['state'] == 'C' and G.degree[node] > max_degree:
                    max_degree = G.degree[node]
                    new_node = node
        elif strategy_type == 'min_degree_C':
            min_degree = float('inf')
            for node in potential_nodes:
                if G.nodes[node]['state'] == 'C' and G.degree[node] < min_degree:
                    min_degree = G.degree[node]
                    new_node = node
        elif strategy_type == 'probabilistic':
            if random.random() < p:
                cooperators = [node for node in potential_nodes if G.nodes[node]['state'] == 'C']
                new_node = random.choice(cooperators) if cooperators else random.choice(potential_nodes)
            else:
                new_node = random.choice(potential_nodes)
        elif strategy_type == 'random':
            if potential_nodes:
                new_node = random.choice(potential_nodes)
        if new_node is None and potential_nodes:
            new_node = random.choice(potential_nodes)
        if new_node:
            G.add_edge(c_node, new_node)
            for neighbor in G.neighbors(c_node):
                potential_edge = tuple(sorted([c_node, neighbor]))
                if G.nodes[c_node]['state'] != G.nodes[neighbor]['state'] and potential_edge not in discordant_edges:
                    discordant_edges.append(potential_edge)
                elif G.nodes[c_node]['state'] == G.nodes[neighbor]['state'] and potential_edge in discordant_edges:
                    discordant_edges.remove(potential_edge)
    return discordant_edges

def simulate(G, alpha, u, w, max_times, strategy_type, p, threshold):
    """
    Simulate network evolution until max iterations or no discordant edges remain.
    Args:
        G (networkx.Graph): The graph to simulate.
        alpha (float): Sensitivity parameter for strategy update.
        u (float): Cost-to-benefit ratio.
        w (float): Probability of strategy update versus rewiring.
        max_times (int): Maximum number of iterations.
        strategy_type (str): Rewiring strategy.
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy.
    """
    iteration = 0
    discordant_edges = [(i, j) for i, j in G.edges() if G.nodes[i]['state'] != G.nodes[j]['state']]
    while iteration < max_times and any(G.nodes[i]['state'] != G.nodes[j]['state'] for i, j in G.edges()):
        if random.random() < w:
            update_strategy(G, alpha, u, discordant_edges)
        else:
            rewire(G, discordant_edges, strategy_type, p, threshold)
        iteration += 1

def calculate_cooperation_fraction(G):
    """
    Calculate the fraction of cooperating nodes in the network.
    Args:
        G (networkx.Graph): The graph with nodes and their states.
    Returns:
        float: Fraction of nodes that are cooperators.
    """
    cooperators = sum(1 for node in G.nodes() if G.nodes[node]['state'] == 'C')
    return cooperators / len(G.nodes())

def simulate_and_average_cooperation(N, M, initial_defector_ratio, alpha, u, w, simulations, max_times, strategy_type, p, threshold):
    """
    Run multiple simulations and return the average fraction of cooperators.
    Args:
        N (int): Number of nodes.
        M (int): Number of edges.
        initial_defector_ratio (float): Initial fraction of defectors.
        alpha (float): Sensitivity parameter for strategy update.
        u (float): Cost-to-benefit ratio.
        w (float): Probability of strategy update versus rewiring.
        simulations (int): Number of simulations to run.
        max_times (int): Maximum iterations per simulation.
        strategy_type (str): Rewiring strategy.
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy.
    Returns:
        float: Average fraction of cooperators across simulations.
    """
    total_cooperation = 0
    for _ in range(simulations):
        G = initialize_network(N, M, initial_defector_ratio)
        simulate(G, alpha, u, w, max_times, strategy_type, p, threshold)
        total_cooperation += calculate_cooperation_fraction(G)
    return total_cooperation / simulations

def run_simulation_for_different_w(w_values, N, M, initial_defector_ratio, alpha, u, simulations, max_times, strategy_types, p, threshold):
    """
    Run simulations for different w values, a fixed u value, and strategy types, returning results in a dictionary.
    Args:
        w_values (list): List of strategy updating probabilities to test.
        N (int): Number of nodes.
        M (int): Number of edges.
        initial_defector_ratio (float): Initial fraction of defectors.
        alpha (float): Sensitivity parameter for strategy update.
        u (float): Fixed cost-to-benefit ratio.
        simulations (int): Number of simulations per condition.
        max_times (int): Maximum iterations per simulation.
        strategy_types (list): List of rewiring strategies to test.
        p (float): Probability for probabilistic rewiring.
        threshold (float): Threshold for hybrid strategy.
    Returns:
        dict: Dictionary mapping (strategy_type, u) to a list of cooperation fractions for each w.
    """
    results = {}
    start_time = time.time()
    for strategy_type in strategy_types:
        for u_val in u:
            cooperation_fractions = []
            for w in w_values:
                avg_cooperation = simulate_and_average_cooperation(
                    N, M, initial_defector_ratio, alpha, u_val, w, simulations, max_times, strategy_type, p, threshold
                )
                cooperation_fractions.append(avg_cooperation)
            results[(strategy_type, u_val)] = cooperation_fractions
    print(f"Total time for all simulations: {time.time() - start_time:.2f} seconds")
    return results

def plot_multiple_cooperation_vs_w(w_values, results):
    """
    Plot the fraction of cooperators versus strategy updating probability w for different strategies and u values.
    Args:
        w_values (list): List of strategy updating probabilities.
        results (dict): Dictionary mapping (strategy_type, u) to a list of cooperation fractions.
    """
    for (strategy_type, u), cooperation_fractions in results.items():
        plt.plot(w_values, cooperation_fractions, marker='o', label=f'{strategy_type}, u={u}')
    plt.xlabel('Strategy Updating Probability, w')  # Label for x-axis
    plt.ylabel('Fraction of cooperators')  # Label for y-axis
    plt.ylim(0, 1)  # Set y-axis limits
    plt.legend()  # Display legend
    plt.show()  # Display the plot

# -------------------------------
# Simulation example parameters
if __name__ == "__main__":
    N = 10000  # Number of nodes
    M = 50000   # Number of edges
    initial_defector_ratio = 0.5  # Initial fraction of defectors
    alpha = 30                  # Sensitivity parameter for strategy update
    u = [0.2]                   # Fixed cost-to-benefit ratio
    w_values = np.linspace(0, 1, 51)  # Range of strategy updating probabilities (51 points from 0 to 1)
    simulations = 1             # Number of simulation runs per condition
    max_times = 10**12          # Maximum iterations per simulation
    strategy_types = ['random']  # Rewiring strategy to test
    p = 1                       # Probability for probabilistic rewiring
    threshold = 0.5             # Threshold for hybrid strategy

    # Run simulations for different w values, a fixed u value, and strategies
    results = run_simulation_for_different_w( w_values, N, M, initial_defector_ratio, alpha, u, simulations, max_times, strategy_types, p, threshold)
    print(results)
    plot_multiple_cooperation_vs_w(w_values, results)